# 参考
- https://github.com/gaussic/text-classification mxnet(gluon)/pytorch实现
- https://mxnet.incubator.apache.org/tutorials/nlp/cnn.html mxnet(sym scratch) 

# load Data

In [1]:
from __future__ import print_function

from collections import Counter
import itertools
import numpy as np
import re

try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen
    
def clean_str(string):
    """
    Tokenization/string cleaning.
    Original from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    
    return string.strip().lower()

def download_sentences(url):
    """
    Download sentences from specified URL. 
    
    Strip trailing newline, convert to Unicode.
    """
    
    remote_file = urlopen(url)
    return [line.decode('Latin1').strip() for line in remote_file.readlines()]
    
def load_data_and_labels():
    """
    Loads polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """

    positive_examples = download_sentences('https://raw.githubusercontent.com/yoonkim/CNN_sentence/master/rt-polarity.pos')
    negative_examples = download_sentences('https://raw.githubusercontent.com/yoonkim/CNN_sentence/master/rt-polarity.neg')
    
    # Tokenize
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent).split(" ") for sent in x_text]

    # Generate labels
    positive_labels = [1 for _ in positive_examples]
    negative_labels = [0 for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return x_text, y


def pad_sentences(sentences, padding_word=""):
    """
    Pads all sentences to be the length of the longest sentence.
    Returns padded sentences.
    """
    sequence_length = max(len(x) for x in sentences)
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = sequence_length - len(sentence)
        new_sentence = sentence + [padding_word] * num_padding
        padded_sentences.append(new_sentence)
        
    return padded_sentences


def build_vocab(sentences):
    """
    Builds a vocabulary mapping from token to index based on the sentences.
    Returns vocabulary mapping and inverse vocabulary mapping.
    """
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    
    return vocabulary, vocabulary_inv


def build_input_data(sentences, labels, vocabulary):
    """
    Maps sentences and labels to vectors based on a vocabulary.
    """
    x = np.array([
            [vocabulary[word] for word in sentence]
            for sentence in sentences])
    y = np.array(labels)
    
    return x, y

"""
Loads and preprocesses data for the MR dataset.
Returns input vectors, labels, vocabulary, and inverse vocabulary.
"""
# Load and preprocess data
sentences, labels = load_data_and_labels()
sentences_padded = pad_sentences(sentences)
vocabulary, vocabulary_inv = build_vocab(sentences_padded)
x, y = build_input_data(sentences_padded, labels, vocabulary)

vocab_size = len(vocabulary)

# randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# split train/test set
# there are a total of 10662 labeled examples to train on
x_train, x_test = x_shuffled[:-1000], x_shuffled[-1000:]
y_train, y_test = y_shuffled[:-1000], y_shuffled[-1000:]

sentence_size = x_train.shape[1]

print('Train/Test split: %d/%d' % (len(y_train), len(y_test)))
print('train shape:', x_train.shape)
print('test shape:', x_test.shape)
print('vocab_size', vocab_size)
print('sentence max words', sentence_size)

Train/Test split: 9662/1000
train shape: (9662, 56)
test shape: (1000, 56)
vocab_size 18766
sentence max words 56


# gen train test set 

In [2]:
from mxnet import gluon
batch_size = 50
dataset_train = gluon.data.ArrayDataset(x_train.astype('float32'), y_train.astype('float32'))
train_data = gluon.data.DataLoader(dataset_train, batch_size, shuffle=True)
dataset_test = gluon.data.ArrayDataset(x_test.astype('float32'), y_test.astype('float32'))
test_data = gluon.data.DataLoader(dataset_test, batch_size, shuffle=True)
for data, label in test_data:
    print(data, label)
    break

/home/work/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/optimizer.py:136: UserWarning: WARNING: New optimizer mxnet.optimizer.NAG is overriding existing optimizer mxnet.optimizer.NAG
  Optimizer.opt_registry[name].__name__))



[[  1.00000000e+00   1.80000000e+01   7.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  2.50000000e+01   4.95000000e+02   1.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  9.80000000e+01   8.41000000e+02   3.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  1.70000000e+01   2.54800000e+03   5.29000000e+02 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.00000000e+00   2.66000000e+02   1.10000000e+01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  8.42200000e+03   7.00000000e+00   2.00000000e+01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
<NDArray 50x56 @cpu(0)> 
[ 1.  0.  1.  1.  1.  1.  0.  1.  0.  0.  1.  1.  0.  1.  0.  1.  0.  0.
  1.  1.  0.  0.  1.  0.  1.  0.  0.  1.  1.  0.  0.  0.  1.  1.  1.  1.
  1.  0.  0.  1.  1.  1.  1.  0.  1.  1.  1.  1.  0.  1.]
<NDArray 50 @cpu(0)>


In [81]:
from mxnet.gluon import Block, nn
from mxnet import ndarray as nd

# 自定义网络结构 mxnet.gluon.Block
# https://mxnet.incubator.apache.org/tutorials/gluon/gluon.html
class Conv_Max_Pooling(Block):
    def __init__(self, filter_size, num_filter, **kwargs):
        super(Conv_Max_Pooling, self).__init__(**kwargs)
        
        self.filter_size = filter_size
        self.num_filter = num_filter
        with self.name_scope():
            self.conv = nn.Conv2D(self.num_filter, self.filter_size, activation='relu')
            self.max_pooling = nn.GlobalMaxPool2D()
    def forward(self, x):
        x = self.conv(x)
        x = self.max_pooling(x)
        return x
    
class TextCNN(Block):
    def __init__(self, vocab_size, num_embed, sentence_size, num_label, num_filter, filter_list=[3, 4, 5], dropout_rate=0.5, **kwargs):
        self.vocab_size = vocab_size
        self.num_embed = num_embed
        self.sentence_size = sentence_size
        self.num_filter = num_filter
        self.filter_list = filter_list
        self.num_label = num_label
        self.dropout_rate = dropout_rate
        
        super(TextCNN, self).__init__(**kwargs)
        with self.name_scope():
            self.embed = nn.Embedding(self.vocab_size, self.num_embed)
            print(self.filter_list)
            self.conv_max_pooling0 = Conv_Max_Pooling((self.filter_list[0], self.num_embed), self.num_filter)
            self.conv_max_pooling1 = Conv_Max_Pooling((self.filter_list[1], self.num_embed), self.num_filter)
            self.conv_max_pooling2 = Conv_Max_Pooling((self.filter_list[2], self.num_embed), self.num_filter)
            self.dropout = nn.Dropout(self.dropout_rate)
            self.dense = nn.Dense(self.num_label)
                
    def forward(self, x):
        x = self.embed(x)
        x = x.reshape(shape=(x.shape[0], 1, self.sentence_size, self.num_embed)) # batch-num_filter-y-x
        x = nd.Concat(self.conv_max_pooling0(x), self.conv_max_pooling1(x), self.conv_max_pooling2(x))
        x = self.dropout(x)
        x = self.dense(x)

        return x

import utilstmp
optimizer = 'adam'
ctx = utilstmp.try_gpu(device_id=1)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
net = TextCNN(vocab_size, 300, sentence_size, 2, 100, [3, 4, 5])
net.collect_params().initialize(ctx=ctx)

trainer = gluon.Trainer(net.collect_params(), optimizer, {'learning_rate': 1e-3, 'wd': 1e-3})
utilstmp.train(train_data, test_data, net, softmax_cross_entropy, trainer, ctx, num_epochs=10)

[3, 4, 5]
Start training on  gpu(1)
Epoch 0. Loss: 0.647, Train acc 0.61, Test acc 0.70, Time 3.1 sec
Epoch 1. Loss: 0.498, Train acc 0.76, Test acc 0.76, Time 2.8 sec
Epoch 2. Loss: 0.405, Train acc 0.82, Test acc 0.76, Time 2.8 sec
Epoch 3. Loss: 0.332, Train acc 0.86, Test acc 0.76, Time 2.8 sec
Epoch 4. Loss: 0.268, Train acc 0.89, Test acc 0.76, Time 2.8 sec
Epoch 5. Loss: 0.214, Train acc 0.92, Test acc 0.76, Time 2.8 sec
Epoch 6. Loss: 0.174, Train acc 0.94, Test acc 0.76, Time 2.8 sec
Epoch 7. Loss: 0.134, Train acc 0.96, Test acc 0.75, Time 2.8 sec
Epoch 8. Loss: 0.115, Train acc 0.96, Test acc 0.76, Time 2.8 sec
Epoch 9. Loss: 0.101, Train acc 0.97, Test acc 0.74, Time 2.8 sec


In [79]:
#参数梯度值
for k, v in net.collect_params().items():
    print(k,v.grad())

textcnn22_embedding0_weight 
[[ -2.16775152e-20  -8.45335324e-18   1.07531599e-19 ...,  -4.09896912e-17
   -6.47573771e-21   7.46259436e-18]
 [  2.79077007e-18   3.87180712e-17   7.88730693e-18 ...,  -6.26867776e-17
   -1.66625674e-17   9.58366329e-18]
 [ -1.10246600e-20   8.31876550e-17   8.02886814e-18 ...,  -1.71907487e-17
    2.79562542e-18  -3.94553472e-17]
 ..., 
 [ -2.60754604e-33  -1.98798752e-32  -1.37415478e-31 ...,   2.17013844e-34
    8.38477417e-32   1.10414588e-32]
 [ -9.76931314e-33  -4.18111081e-34  -3.47800096e-32 ...,   9.16216916e-37
    1.11180496e-33   1.63121044e-32]
 [  1.40164313e-35  -2.32587102e-26   1.40272476e-32 ...,   1.38226271e-33
   -3.61968300e-32   4.91523853e-32]]
<NDArray 18766x300 @gpu(1)>
textcnn22_conv_max_pooling0_conv0_weight 
[[[[  8.70877640e-21   3.68553320e-19   8.72432045e-20 ...,
     -4.79607720e-21  -5.55611724e-21  -1.21868664e-18]
   [ -3.29547136e-26  -3.34625771e-20  -3.11294689e-20 ...,
     -2.56942590e-20  -9.92831078e-19   1.794

In [80]:
#参数值
for k, v in net.collect_params().items():
    print(k,v.data())

textcnn22_embedding0_weight 
[[ -2.04428916e-20   1.44734161e-16   4.82900933e-20 ...,  -2.01231784e-17
   -7.05945662e-21  -3.06642719e-18]
 [  3.66353578e-18   1.89430117e-17   6.87378321e-18 ...,  -5.03504376e-17
   -1.01702237e-17   1.46943309e-17]
 [ -5.49032165e-21   8.24906131e-17   1.21735294e-17 ...,  -2.02416042e-17
    5.16589306e-18  -4.04359566e-17]
 ..., 
 [ -2.40293692e-33  -1.53736941e-32  -9.69563022e-33 ...,  -1.36691855e-34
    1.64039790e-31   1.47661780e-32]
 [ -6.49538799e-33  -1.68760742e-34  -3.50373812e-32 ...,   1.33113615e-36
    8.57660331e-34   7.75360923e-33]
 [ -1.76180660e-36  -1.85803675e-26   2.81463835e-32 ...,   1.50435276e-33
   -4.30209685e-32  -2.71451327e-32]]
<NDArray 18766x300 @gpu(1)>
textcnn22_conv_max_pooling0_conv0_weight 
[[[[  9.25443934e-21   5.74257526e-19   1.27392044e-19 ...,
      2.63054873e-20  -8.61719927e-21  -1.58647606e-18]
   [ -3.59572292e-25  -3.64921584e-20   7.18301284e-20 ...,
      1.92448520e-19  -7.95275533e-19  -2.498

In [42]:
# 写网络用的此时数据，为确定每层的输出情况
import mxnet
data, label = 0, 0
for d, l in train_data:
    data, label = d, l
    break
# data = data.copyto(ctx)
# label = label.copyto(ctx)
data, label
data = data.copyto(mxnet.gpu(1))

In [5]:
from mxnet import gluon
from mxnet.gluon import nn

import sys
sys.path.append('.')
from mxnet import ndarray as nd
from mxnet import autograd
import utilstmp
import time

def net_structure(num_hidden, num_outputs, ctx, num_embed):
    net = gluon.nn.Sequential()
    num_embed = num_embed # dimensions to embed words into
    filter_size = 3
    
    with net.name_scope():
        net.add(
            nn.Embedding(vocab_size, num_embed),
            nn.Conv1D(1, 3),
            nn.GlobalMaxPool1D(),
            nn.Dropout(0.5),
            nn.Dense(num_outputs)
        )
        print(net)
        print(net.collect_params())
    net.initialize(ctx=ctx)
    return net

def mlp(optimizer='sgd', num_outputs=2, num_hidden=256, weight_scale=.01, learning_rate=0.0005, 
        num_epoch=10, batch_size=50, num_embed=300):
    batch_size = batch_size

    ctx = utilstmp.try_gpu(device_id=1)
    net = net_structure(num_hidden=num_hidden, num_outputs=num_outputs, ctx=ctx, num_embed=num_embed)
    loss = gluon.loss.SoftmaxCrossEntropyLoss()
    
    trainer = gluon.Trainer(net.collect_params(), optimizer, {'learning_rate': learning_rate})
    utilstmp.train(train_data, test_data, net, loss, trainer, ctx, num_epochs=num_epoch)

# https://mxnet.incubator.apache.org/tutorials/nlp/cnn.html mxnet(sym scratch)

In [7]:
from __future__ import print_function

from collections import Counter
import itertools
import numpy as np
import re

try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen
    
def clean_str(string):
    """
    Tokenization/string cleaning.
    Original from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    
    return string.strip().lower()

def download_sentences(url):
    """
    Download sentences from specified URL. 
    
    Strip trailing newline, convert to Unicode.
    """
    
    remote_file = urlopen(url)
    return [line.decode('Latin1').strip() for line in remote_file.readlines()]
    
def load_data_and_labels():
    """
    Loads polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """

    positive_examples = download_sentences('https://raw.githubusercontent.com/yoonkim/CNN_sentence/master/rt-polarity.pos')
    negative_examples = download_sentences('https://raw.githubusercontent.com/yoonkim/CNN_sentence/master/rt-polarity.neg')
    
    # Tokenize
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent).split(" ") for sent in x_text]

    # Generate labels
    positive_labels = [1 for _ in positive_examples]
    negative_labels = [0 for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return x_text, y


def pad_sentences(sentences, padding_word=""):
    """
    Pads all sentences to be the length of the longest sentence.
    Returns padded sentences.
    """
    sequence_length = max(len(x) for x in sentences)
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = sequence_length - len(sentence)
        new_sentence = sentence + [padding_word] * num_padding
        padded_sentences.append(new_sentence)
        
    return padded_sentences


def build_vocab(sentences):
    """
    Builds a vocabulary mapping from token to index based on the sentences.
    Returns vocabulary mapping and inverse vocabulary mapping.
    """
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    
    return vocabulary, vocabulary_inv


def build_input_data(sentences, labels, vocabulary):
    """
    Maps sentences and labels to vectors based on a vocabulary.
    """
    x = np.array([
            [vocabulary[word] for word in sentence]
            for sentence in sentences])
    y = np.array(labels)
    
    return x, y

"""
Loads and preprocesses data for the MR dataset.
Returns input vectors, labels, vocabulary, and inverse vocabulary.
"""
# Load and preprocess data
sentences, labels = load_data_and_labels()
sentences_padded = pad_sentences(sentences)
vocabulary, vocabulary_inv = build_vocab(sentences_padded)
x, y = build_input_data(sentences_padded, labels, vocabulary)

vocab_size = len(vocabulary)

# randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# split train/dev set
# there are a total of 10662 labeled examples to train on
x_train, x_dev = x_shuffled[:-1000], x_shuffled[-1000:]
y_train, y_dev = y_shuffled[:-1000], y_shuffled[-1000:]

sentence_size = x_train.shape[1]

print('Train/Dev split: %d/%d' % (len(y_train), len(y_dev)))
print('train shape:', x_train.shape)
print('dev shape:', x_dev.shape)
print('vocab_size', vocab_size)
print('sentence max words', sentence_size)

import mxnet as mx
import sys,os

'''
Define batch size and the place holders for network inputs and outputs
'''

batch_size = 50
print('batch size', batch_size)

input_x = mx.sym.Variable('data') # placeholder for input data
input_y = mx.sym.Variable('softmax_label') # placeholder for output label


'''
Define the first network layer (embedding)
'''

# create embedding layer to learn representation of words in a lower dimensional subspace (much like word2vec)
num_embed = 300 # dimensions to embed words into
print('embedding dimensions', num_embed)

embed_layer = mx.sym.Embedding(data=input_x, input_dim=vocab_size, output_dim=num_embed, name='vocab_embed')

# reshape embedded data for next layer
conv_input = mx.sym.Reshape(data=embed_layer, target_shape=(batch_size, 1, sentence_size, num_embed))

# create convolution + (max) pooling layer for each filter operation
filter_list=[3, 4, 5] # the size of filters to use
print('convolution filters', filter_list)

num_filter=100
pooled_outputs = []
for filter_size in filter_list:
    convi = mx.sym.Convolution(data=conv_input, kernel=(filter_size, num_embed), num_filter=num_filter)
    relui = mx.sym.Activation(data=convi, act_type='relu')
    pooli = mx.sym.Pooling(data=relui, pool_type='max', kernel=(sentence_size - filter_size + 1, 1), stride=(1, 1))
    pooled_outputs.append(pooli)

# combine all pooled outputs
total_filters = num_filter * len(filter_list)
concat = mx.sym.Concat(*pooled_outputs, dim=1)

# reshape for next layer
h_pool = mx.sym.Reshape(data=concat, target_shape=(batch_size, total_filters))

# dropout layer
dropout = 0.5
print('dropout probability', dropout)

if dropout > 0.0:
    h_drop = mx.sym.Dropout(data=h_pool, p=dropout)
else:
    h_drop = h_pool
    
# fully connected layer
num_label = 2

cls_weight = mx.sym.Variable('cls_weight')
cls_bias = mx.sym.Variable('cls_bias')

fc = mx.sym.FullyConnected(data=h_drop, weight=cls_weight, bias=cls_bias, num_hidden=num_label)

# softmax output
sm = mx.sym.SoftmaxOutput(data=fc, label=input_y, name='softmax')

# set CNN pointer to the "back" of the network
cnn = sm

from collections import namedtuple
import math
import time

# Define the structure of our CNN Model (as a named tuple)
CNNModel = namedtuple("CNNModel", ['cnn_exec', 'symbol', 'data', 'label', 'param_blocks'])

# Define what device to train/test on
ctx = mx.gpu(0)
# If you have no GPU on your machine change this to
# ctx = mx.cpu(0)

arg_names = cnn.list_arguments()

input_shapes = {}
input_shapes['data'] = (batch_size, sentence_size)

arg_shape, out_shape, aux_shape = cnn.infer_shape(**input_shapes)
arg_arrays = [mx.nd.zeros(s, ctx) for s in arg_shape]
args_grad = {}
for shape, name in zip(arg_shape, arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    args_grad[name] = mx.nd.zeros(shape, ctx)

cnn_exec = cnn.bind(ctx=ctx, args=arg_arrays, args_grad=args_grad, grad_req='add')

param_blocks = []
arg_dict = dict(zip(arg_names, cnn_exec.arg_arrays))
initializer = mx.initializer.Uniform(0.1)
for i, name in enumerate(arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    initializer(mx.init.InitDesc(name), arg_dict[name])

    param_blocks.append( (i, arg_dict[name], args_grad[name], name) )

data = cnn_exec.arg_dict['data']
label = cnn_exec.arg_dict['softmax_label']

cnn_model= CNNModel(cnn_exec=cnn_exec, symbol=cnn, data=data, label=label, param_blocks=param_blocks)

'''
Train the cnn_model using back prop
'''

optimizer = 'rmsprop'
max_grad_norm = 5.0
learning_rate = 0.0005
epoch = 10

print('optimizer', optimizer)
print('maximum gradient', max_grad_norm)
print('learning rate (step size)', learning_rate)
print('epochs to train for', epoch)

# create optimizer
opt = mx.optimizer.create(optimizer)
opt.lr = learning_rate

updater = mx.optimizer.get_updater(opt)

# For each training epoch
for iteration in range(epoch):
    tic = time.time()
    num_correct = 0
    num_total = 0

    # Over each batch of training data
    for begin in range(0, x_train.shape[0], batch_size):
        batchX = x_train[begin:begin+batch_size]
        batchY = y_train[begin:begin+batch_size]
        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.label[:] = batchY

        # forward
        cnn_model.cnn_exec.forward(is_train=True)

        # backward
        cnn_model.cnn_exec.backward()

        # eval on training data
        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)

        # update weights
        norm = 0
        for idx, weight, grad, name in cnn_model.param_blocks:
            grad /= batch_size
            l2_norm = mx.nd.norm(grad).asscalar()
            norm += l2_norm * l2_norm

        norm = math.sqrt(norm)
        for idx, weight, grad, name in cnn_model.param_blocks:
            if norm > max_grad_norm:
                grad *= (max_grad_norm / norm)

            updater(idx, grad, weight)

            # reset gradient to zero
            grad[:] = 0.0

    # Decay learning rate for this epoch to ensure we are not "overshooting" optima
    if iteration % 50 == 0 and iteration > 0:
        opt.lr *= 0.5
        print('reset learning rate to %g' % opt.lr)

    # End of training loop for this epoch
    toc = time.time()
    train_time = toc - tic
    train_acc = num_correct * 100 / float(num_total)

    # Saving checkpoint to disk
    if (iteration + 1) % 10 == 0:
        prefix = 'cnn'
        cnn_model.symbol.save('./%s-symbol.json' % prefix)
        save_dict = {('arg:%s' % k) : v  for k, v in cnn_model.cnn_exec.arg_dict.items()}
        save_dict.update({('aux:%s' % k) : v for k, v in cnn_model.cnn_exec.aux_dict.items()})
        param_name = './%s-%04d.params' % (prefix, iteration)
        mx.nd.save(param_name, save_dict)
        print('Saved checkpoint to %s' % param_name)


    # Evaluate model after this epoch on dev (test) set
    num_correct = 0
    num_total = 0

    # For each test batch
    for begin in range(0, x_dev.shape[0], batch_size):
        batchX = x_dev[begin:begin+batch_size]
        batchY = y_dev[begin:begin+batch_size]

        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.cnn_exec.forward(is_train=False)

        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)

    dev_acc = num_correct * 100 / float(num_total)
    print('Iter [%d] Train: Time: %.3fs, Training Accuracy: %.3f \
            --- Dev Accuracy thus far: %.3f' % (iteration, train_time, train_acc, dev_acc))

<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence 

<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence 

<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence 

<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence 

<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence 

...

<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence 

<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence 

<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence 

<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence \)
<input>:31: DeprecationWarning: invalid escape sequence \?
<input>:29: DeprecationWarning: invalid escape sequence \(
<input>:30: DeprecationWarning: invalid escape sequence 

Train/Dev split: 9662/1000
train shape: (9662, 56)
dev shape: (1000, 56)
vocab_size 18766
sentence max words 56
batch size 50
embedding dimensions 300
convolution filters [3, 4, 5]
dropout probability 0.5
optimizer rmsprop
maximum gradient 5.0
learning rate (step size) 0.0005
epochs to train for 10
Iter [0] Train: Time: 3.059s, Training Accuracy: 56.705             --- Dev Accuracy thus far: 62.500
Iter [1] Train: Time: 2.907s, Training Accuracy: 71.461             --- Dev Accuracy thus far: 69.800
Iter [2] Train: Time: 2.862s, Training Accuracy: 81.876             --- Dev Accuracy thus far: 74.700
Iter [3] Train: Time: 2.857s, Training Accuracy: 87.762             --- Dev Accuracy thus far: 76.200
Iter [4] Train: Time: 2.861s, Training Accuracy: 91.720             --- Dev Accuracy thus far: 77.200
Iter [5] Train: Time: 2.863s, Training Accuracy: 94.363             --- Dev Accuracy thus far: 77.200
Iter [6] Train: Time: 2.866s, Training Accuracy: 96.114             --- Dev Accuracy thu

In [24]:
for l in cnn_model.param_blocks:
    for v in l:
        if(type(v) != int and type(v) != str):
            print(v.shape)

(18766, 300)
(18766, 300)
(100, 1, 3, 300)
(100, 1, 3, 300)
(100,)
(100,)
(100, 1, 4, 300)
(100, 1, 4, 300)
(100,)
(100,)
(100, 1, 5, 300)
(100, 1, 5, 300)
(100,)
(100,)
(2, 300)
(2, 300)
(2,)
(2,)
